In [1]:
#Start up Spark (only needs to be done once)
%reset
import os
import pyspark.sql
import sys
from pyspark.sql.functions import sqrt
from pyspark.sql.functions import cosh
from pyspark.sql.functions import cos
import uproot
#from pyspark.sql import SparkSession

#!export KRB5CCNAME=FILE:/tmp/${USER}_krb
#!kinit -c /tmp/${USER}_krb

session = pyspark.sql.SparkSession.builder \
    .master(
        #"spark://10.64.22.66:7077"
        "local"
    ) \
    .appName(# Name of your application in the dashboard/UI
             "Zpeak"
            ) \
    .config(# Tell Spark to load some extra libraries from Maven (the Java repository)
            'spark.jars.packages',
            'org.diana-hep:spark-root_2.11:0.1.16,org.diana-hep:histogrammar-sparksql_2.11:1.0.4'
            )\
    .config('spark.cores.max,3'
            ) \
    .getOrCreate()
    
sc = session.sparkContext
sqlContext = session
!hdfs dfs -ls hdfs://10.64.22.72:9000/NANO_Prod

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
Found 8 items
-rw-r--r--   2 shoh supergroup  463520454 2018-02-06 12:58 hdfs://10.64.22.72:9000/NANO_Prod/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_ext1-v2.root
-rw-r--r--   2 shoh supergroup  175945343 2018-02-06 12:58 hdfs://10.64.22.72:9000/NANO_Prod/SingleElectronRun2016C-03Feb2017-v1.root
-rw-r--r--   2 shoh supergroup  194359897 2018-02-06 12:58 hdfs://10.64.22.72:9000/NANO_Prod/SingleMuonRun2016C-03Feb2017-v1.root
-rw-r--r--   2 shoh supergroup  560222724 2018-02-06 12:58 hdfs://10.64.22.72:9000/NANO_Prod/TT_TuneCUETP8M2T4_13TeV-powheg-pythia8-v1.root
-rw-r--r--   2 shoh supergroup  276454242 2018-02-06 12:58 hdfs://10.64.22.72:9000/NANO_Prod/WW_TuneCUETP8M1_13TeV-pythia8-v1.root
-rw-r--r--   2 shoh supergroup  469017530 2018-02-06 12:58 hdfs://10.64.22.72:9000/NANO_Prod/WZTo2L2Q_13TeV_amcatnloFXFX_madspin_pythia8-v1.root
-rw-r--r--   2 shoh supergroup  340699069 2018-02-06 12:59 hdfs://10.64.22.72:900

In [2]:
filename="hdfs://10.64.22.72:9000/NANO_Prod/DYJetsToLL_M-50_TuneCUETP8M1_13TeV-madgraphMLM-pythia8_ext1-v2.root"
df = sqlContext.read.format("org.dianahep.sparkroot") \
    .option("tree","Events") \
    .load(filename)
df.count()

549790

In [3]:
df.printSchema()
#df.show()


root
 |-- run: integer (nullable = true)
 |-- luminosityBlock: integer (nullable = true)
 |-- event: long (nullable = true)
 |-- CaloMET_phi: float (nullable = true)
 |-- CaloMET_pt: float (nullable = true)
 |-- CaloMET_sumEt: float (nullable = true)
 |-- nElectron: integer (nullable = true)
 |-- Electron_deltaEtaSC: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Electron_dr03EcalRecHitSumEt: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Electron_dr03HcalDepth1TowerSumEt: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Electron_dr03TkSumPt: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Electron_dxy: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Electron_dxyErr: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Electron_dxyErr_ugo: array (nullable = true)
 |    |-- element: float (containsNull = true)
 |-- Electron_d

In [4]:
header = list(df.columns)
muonAttr = [muon for muon in header if 'Muon' in muon]
dsMuons = df.select(muonAttr)
#dsMuons.write.save("file:/home/shoh/jupyNotebook/mllnano.parquet", format="parquet")
#dsMuons.write.parquet("file:/home/shoh/jupyNotebook/mllnano.parquet")
parquetDF = sqlContext.read.parquet("file:/home/shoh/jupyNotebook/mllnano.parquet")
display(parquetDF)


DataFrame[Jet_nMuons: array<int>, nMuon: int, Muon_dxy: array<float>, Muon_dxyErr: array<float>, Muon_dxyErr_ugo: array<float>, Muon_dxy_ugo: array<float>, Muon_dz: array<float>, Muon_dzErr: array<float>, Muon_eta: array<float>, Muon_ip3d: array<float>, Muon_ip3d_ugo: array<float>, Muon_mass: array<float>, Muon_miniPFRelIso_all: array<float>, Muon_miniPFRelIso_chg: array<float>, Muon_pfRelIso03_all: array<float>, Muon_pfRelIso03_chg: array<float>, Muon_pfRelIso04_all: array<float>, Muon_phi: array<float>, Muon_pt: array<float>, Muon_ptErr: array<float>, Muon_segmentComp: array<float>, Muon_sip3d: array<float>, Muon_sip3d_ugo: array<float>, Muon_mvaTTH: array<float>, Muon_charge: array<int>, Muon_jetIdx: array<int>, Muon_nStations: array<int>, Muon_nTrackerLayers: array<int>, Muon_pdgId: array<int>, Muon_tightCharge: array<int>, Muon_highPtId: array<tinyint>, Muon_isPFcand: array<boolean>, Muon_mediumId: array<boolean>, Muon_softId: array<boolean>, Muon_tightId: array<boolean>, Muon_gen

In [13]:
import oamap.source.parquet
physicsObjects = oamap.source.parquet.open("/home/shoh/jupyNotebook/mllnano.parquet/part-00000-ab9dd62b-b1e7-435f-b0e9-e443f8497356.snappy.parquet")
#physicsObjects
#import pandas as pd
#import pyarrow.parquet as pq
#df_mll = pq.ParquetDataset("/home/shoh/jupyNotebook/mllnano.parquet").read().to_pandas()

ThriftParserError: Dead including on /home/shoh/.local/lib/python2.7/site-packages/oamap/source/_fastparquet/parquet.thrift